* Always export as PDF or SVG rather than PNG
* sometimes it’s easier to do customisation manually in the end (width, how close are bars together, etc etc etc – it’s easier in plotly than in matplotlib, but even easier in inkscape or similar)


# Hier muss die Syntax von pandas geupdated werden!

## Saving figures

In [1]:
import plotly.graph_objs as go
import plotly.offline as pltly
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import colorlover as cl
import seaborn as sns

%matplotlib inline

df = pd.read_csv('./data/time_series_60min_singleindex.csv', 
                 index_col=0, 
                 parse_dates=True, 
                 low_memory=False)

df_wind = df.loc[:, [
    'CZ_wind_generation',
    'DK_wind_offshore_generation',
    'DK_wind_onshore_generation',
    'SE_wind_generation'
]]

df_wind = df_wind.rename(columns={
    k: k.replace('_wind_', ' ').replace('generation', '') for k in df_wind.columns
})

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

### matplotlib

In [ ]:
sns.set_style('ticks')

fig = plt.figure(figsize=(16, 6))
ax0 = plt.subplot(121)
ax1 = plt.subplot(122)

df_wind.resample('1M').mean().loc['2012':'2015'].plot(ax=ax0, colormap='Spectral')
df_wind.resample('12M').mean().loc['2012':'2015'].plot.bar(stacked=True, ax=ax1, colormap='Spectral')

ax0.set_xlabel('Monthly variation')
ax1.set_xlabel('Annual variation')

ax1.set_xticklabels(range(2012, 2016))
plt.xticks(rotation='0')

sns.despine()


# Format is determined by file ending or can be given, e.g. format='svg'

plt.savefig(
    'matplotlib_wind.pdf',
    dpi=600, bbox_inches='tight', pad_inches=0.2
)

### Plotly
This code will open a new browser tab with the plot and will ask to download an SVG image. The HTML file will be saved in your visualisation tutorial folder.

In [ ]:
data = []
colors = cl.scales[str(len(df_wind.columns))]['div']['Spectral']
color_index = 0
for column in df_wind.columns:
    data.append(go.Scatter(x=df_wind.resample('1M').mean()[column].loc['2012':'2015-06'].index,
                           y=df_wind.resample('1M').mean()[column].loc['2012':'2015-06'].values,
                           name=column,
                           xaxis='x1',
                           yaxis='y1',
                           line=dict(color=colors[color_index]),
                           legendgroup=column))
    data.append(go.Bar(x=df_wind.resample('12M').mean()[column].loc['2012':].index,
                       y=df_wind.resample('12M').mean()[column].loc['2012':].values,
                       name=column,
                       xaxis='x2',
                       yaxis='y2',
                       marker=dict(color=colors[color_index]),
                       showlegend=False,
                       legendgroup=column,
                       hoverinfo='y'))
    color_index += 1
layout = dict(barmode='stack', height=400, width=800,
              xaxis1=dict(domain=[0,0.49], title='Monthly variation'),
              xaxis2=dict(domain=[0.51,1], title='Annual variation'),
              yaxis1=dict(range=[0,2500]),
              yaxis2=dict(anchor='x2',range=[0,3600], side='right'),
              legend=dict(orientation='h', xanchor='center',yanchor='bottom', x=0.5, y=1))
fig = go.Figure(data=data, layout=layout)
pltly.plot(fig, filename='plotly_wind_data.html', image='svg')